In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import sys

In [4]:
# Add src directory to path for local imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Local imports
from utils import gpu_grow_memory

In [5]:
gpu_grow_memory()

In [6]:
tfds.disable_progress_bar()

In [7]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

W0620 12:34:53.750726 22296 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow_datasets\core\file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to C:\Users\Michael\tensorflow_datasets\mnist\1.0.0. Subsequent calls will reuse this data.


In [8]:
mnist_train, mnist_test = datasets['train'], datasets['test']

In [35]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.MirroredStrategy(['/GPU:0', '/GPU:1'])

In [34]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [11]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [12]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [13]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [14]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [21]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [23]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [24]:
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [25]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [27]:
# model.fit(train_dataset, epochs=12, callbacks=callbacks)
model.fit(train_dataset, epochs=12, callbacks=None)

Train on None steps
Epoch 1/12


InvalidArgumentError: No OpKernel was registered to support Op 'NcclAllReduce' used by {{node Adam/NcclAllReduce}}with these attrs: [reduction="sum", shared_name="c2", T=DT_FLOAT, num_devices=2]
Registered devices: [CPU, GPU]
Registered kernels:
  <no registered kernels>

	 [[Adam/NcclAllReduce]] [Op:__inference_distributed_function_213019]